# Installs and imports

In [1]:
!pip install spacy

In [2]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 10.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [3]:
import spacy


nlp = spacy.load("ru_core_news_sm")  # загружаем предобученный пайплайн для языка

# Spacy pipelines

[Пайплайн](https://spacy.io/api#architecture-pipeline) состоит из нескольких компонентов, которые последовательно применяются друг за другом. Вот так можно посмотреть из каких компонентов состоит пайплайн:

In [15]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x15c05b290>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x15c05b0b0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x14829f370>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x15dad1fd0>),
 ('lemmatizer', <spacy.lang.ru.lemmatizer.RussianLemmatizer at 0x15c087810>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x15e1a18c0>)]

А вот так можно узнать, какой аттрибут какой компонент приписывает:

In [4]:
nlp.analyze_pipes()

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'morphologizer': {'assigns': ['token.morph', 'token.pos'],
   'requires': [],
   'scores': ['pos_acc', 'morph_acc', 'morph_per_feat'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  'morphologizer': [],
  'parser': [],
  'a

Например, отсюда видно, что частеречный тег и грамматические отношению задаются морфологизатором, а лемма задается лемматизатором.

In [10]:
morphologizer = nlp.get_pipe('morphologizer')

Поэтому, если нас не устраивает результат какого-то компонента можно: </br>
а) заменить его на другой </br>
б) добавить еще один, который исправлять ошибки первого</br>

О том как добавлять компоненты написано [тут](https://spacy.io/usage/processing-pipelines#custom-components)

## Custom components

Например, как-то так можно добавить морфологизатор на основе pymorphy

In [29]:
!pip install pymorphy3

In [32]:
!pip install russian-tagsets

  Preparing metadata (setup.py) ... done
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-py3-none-any.whl size=24619 sha256=8f33d93191819c7f9dda5d61b9b19b7ee49175fa3145dcf9ca2bc48a3fad28c0
  Stored in directory: /Users/viktoriaknazkova/Library/Caches/pip/wheels/47/7b/e4/a364b538d52cbc10811f88365af0f40bc2137d1dffa73d7082
Successfully built russian-tagsets


In [89]:
import pymorphy3
from russian_tagsets import converters
from spacy.language import Language

morph = pymorphy3.MorphAnalyzer(lang='ru')
oc2ud = converters.converter('opencorpora-int', 'ud20')  # чтобы сконвертировать в ud

@Language.component("custom_morphologizer")
def my_component(doc):
    for t in doc:
        parsed = morph.parse(t.text)[0]
        t.lemma_ = parsed.normal_form
        
        oc_tag = str(parsed.tag)
        ud_tag = oc2ud(oc_tag).split(' ')
        t.pos_ = ud_tag[0]
        t.set_morph(ud_tag[1])
    return doc

In [91]:
custom_nlp = spacy.load("ru_core_news_sm", exclude=['morphologizer', 'lemmatizer', 'ner'])
custom_nlp.add_pipe('custom_morphologizer', after='tok2vec')
custom_nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1235a4470>),
 ('custom_morphologizer', <function __main__.my_component(doc)>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x11e71fdf0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x2ad57ce50>)]

In [92]:
ex_text = 'Подкоренное выражение представляет из себя сумму квадрата разности и одночлена четыре а икс. '

In [93]:
for t in custom_nlp(ex_text):
    print(t.text, t.lemma_, t.pos_, t.morph, sep='\t')

Подкоренное	подкоренной	ADJ	Case=Nom|Gender=Neut|Number=Sing
выражение	выражение	NOUN	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing
представляет	представлять	VERB	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
из	из	ADP	
себя	себя	PRON	Case=Acc|Number=Sing
сумму	сумма	NOUN	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
квадрата	квадрат	NOUN	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
разности	разность	NOUN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
и	и	CCONJ	
одночлена	одночлен	NOUN	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
четыре	четыре	NUM	Animacy=Inan|Case=Acc
а	а	CCONJ	
икс	икс	NOUN	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
.	.	PUNCT	


In [94]:
# сравним с оригинальным
for t in nlp(ex_text):
    print(t.text, t.lemma_, t.pos_, t.morph, sep='\t')

Подкоренное	подкоренной	ADJ	Case=Nom|Degree=Pos|Gender=Neut|Number=Sing
выражение	выражение	NOUN	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing
представляет	представлять	VERB	Aspect=Imp|Mood=Ind|Number=Sing|Person=Third|Tense=Pres|VerbForm=Fin|Voice=Act
из	из	ADP	
себя	себя	PRON	Case=Gen
сумму	сумма	NOUN	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
квадрата	квадрат	NOUN	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
разности	разность	NOUN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
и	и	CCONJ	
одночлена	одночлена	VERB	Aspect=Perf|Gender=Fem|Number=Sing|StyleVariant=Short|Tense=Past|VerbForm=Part|Voice=Pass
четыре	четыре	NUM	Case=Nom
а	а	CCONJ	
икс	икс	PROPN	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
.	.	PUNCT	


# Language

В spacy есть специальный контейнер - [Language](https://spacy.io/api/language), который содержит в себе все конфиги, дефолтные значения, лукап таблицы, компоненты пайплайна и т.д.

In [125]:
# Например, предобученная модель, которую мы загружаем через spacy.load создает объект класса нашего языка

nlp.__class__

spacy.lang.ru.Russian

In [126]:
# который в свою очередь является потомком общего класса Language
nlp.__class__.__bases__

(spacy.language.Language,)

Можно [создать](https://spacy.io/usage/linguistic-features#language-subclass) свой собственный класс языка и определить там нужное поведение и аттрибуты. Например, там можно инкапсулировать все кастомные компоненты и добавить нужные лукап таблицы и тд